In [63]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text and covert to lowercase
from nltk.corpus import PlaintextCorpusReader
import os
import sys
wordlists = PlaintextCorpusReader("Nepali_Corpus", '.*txt')
data = wordlists.fileids()[:100]
text = []
for i in data:
    with open(os.path.join("Nepali_Corpus",i)) as file:
        text.append(file.read())
text = "\n".join(text)
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])


Total Characters:  166466
Total Vocab:  91


In [64]:
len(dataX[0])

100

In [65]:
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize


Total Patterns:  166366


In [66]:
X[0]/n_vocab

array([[0.59340659],
       [0.32967033],
       [0.63736264],
       [0.69230769],
       [0.53846154],
       [0.54945055],
       [0.72527473],
       [0.61538462],
       [0.01098901],
       [0.94505495],
       [0.01098901],
       [0.56043956],
       [0.7032967 ],
       [0.62637363],
       [0.8021978 ],
       [0.59340659],
       [0.01098901],
       [0.95604396],
       [0.68131868],
       [0.72527473],
       [0.61538462],
       [0.8021978 ],
       [0.61538462],
       [0.75824176],
       [0.96703297],
       [0.62637363],
       [0.75824176],
       [0.01098901],
       [0.61538462],
       [0.7032967 ],
       [0.62637363],
       [0.7032967 ],
       [0.40659341],
       [0.01098901],
       [0.59340659],
       [0.7032967 ],
       [0.49450549],
       [0.7032967 ],
       [0.01098901],
       [0.53846154],
       [0.40659341],
       [0.7032967 ],
       [0.32967033],
       [0.7032967 ],
       [0.28571429],
       [0.67032967],
       [0.17582418],
       [0.351

In [67]:
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)


In [68]:
X.shape

(166366, 100, 1)

In [69]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
#model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# load the network weights
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
166366/166366 [==============================] - 730s 4ms/step - loss: 3.4115

Epoch 00001: loss improved from inf to 3.41154, saving model to weights-improvement-01-3.4115.hdf5
Epoch 2/20
166366/166366 [==============================] - 692s 4ms/step - loss: 3.2199

Epoch 00002: loss improved from 3.41154 to 3.21993, saving model to weights-improvement-02-3.2199.hdf5
Epoch 3/20
166366/166366 [==============================] - 688s 4ms/step - loss: 3.1118

Epoch 00003: loss improved from 3.21993 to 3.11184, saving model to weights-improvement-03-3.1118.hdf5
Epoch 4/20
166366/166366 [==============================] - 733s 4ms/step - loss: 3.0102

Epoch 00004: loss improved from 3.11184 to 3.01019, saving model to weights-improvement-04-3.0102.hdf5
Epoch 5/20
166366/166366 [==============================] - 681s 4ms/step - loss: 2.9220

Epoch 00005: loss improved from 3.01019 to 2.92203, saving model to weights-improvement-05-2.9220.hdf5
Epoch 6/20
166366/166366 [=============

In [70]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
gen_data = [int_to_char[value] for value in pattern]
gen_data.append("***")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    print(prediction)
    result = int_to_char[index]
    try:
        gen_data.append(int_to_char[result])
    except:
        gen_data.append(" ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

[[5.02890325e-04 3.41372073e-01 5.35153610e-09 4.38852339e-07
  1.34940656e-05 6.42889136e-05 9.41653643e-03 7.08500784e-06
  2.46574700e-07 4.73763926e-08 4.95510761e-08 6.42153136e-11
  1.48622773e-03 4.70284448e-07 4.43830004e-08 5.93023834e-08
  1.13743753e-03 2.32723920e-04 3.52334979e-07 4.05676663e-04
  3.44917906e-04 4.98107438e-05 2.32468150e-03 1.41698329e-05
  2.88326287e-08 2.41357836e-11 1.42274366e-05 4.72900087e-11
  1.05399920e-07 4.55830133e-07 6.77089617e-02 4.57384944e-04
  4.84545220e-04 4.63264769e-06 2.83048394e-05 2.69741082e-04
  1.63223012e-04 1.30102900e-03 5.40675537e-05 7.98047695e-05
  4.16045859e-02 1.02459526e-04 5.13620384e-04 5.52711299e-07
  8.44589726e-04 8.61804038e-02 3.17647681e-03 6.54200045e-03
  4.13053902e-03 1.46523202e-02 1.12550764e-03 1.02613203e-03
  1.27446325e-03 5.29593090e-04 2.04502866e-01 1.02997974e-01
  9.44826007e-03 5.32078259e-02 1.04901344e-02 1.87013554e-03
  3.88574641e-04 1.66024584e-02 1.02958630e-03 3.16194817e-03
  1.1202

[[5.28875462e-05 5.60926534e-02 4.70485055e-08 6.07403927e-05
  1.58134966e-07 8.20798391e-07 4.27495746e-04 2.99573039e-05
  5.62587811e-05 4.81584828e-10 2.86361028e-05 5.01399269e-12
  5.76612016e-04 2.10794766e-04 2.35831158e-06 6.19491347e-09
  2.43972172e-03 4.54062270e-03 4.30615939e-04 2.78309220e-03
  1.36361318e-03 4.32462338e-03 3.85687221e-04 4.49930143e-04
  2.70076780e-05 1.71456141e-10 1.36376396e-02 1.37763290e-11
  2.50797730e-05 1.61290814e-07 8.24434683e-03 1.29496609e-03
  6.05770200e-03 7.73550855e-05 3.40873870e-04 7.15283118e-03
  1.24181237e-03 1.03124855e-02 5.68232281e-05 1.91042405e-02
  4.85065673e-03 1.87524798e-04 2.02142401e-03 5.02313778e-04
  2.71932199e-03 4.67741769e-03 2.46637454e-03 2.51102285e-03
  1.81161449e-03 1.49231926e-02 1.24647813e-02 6.81990804e-03
  2.96253115e-02 1.12572685e-02 7.84306452e-02 3.63572943e-03
  2.24171430e-02 2.63049956e-02 5.90860518e-03 3.73051991e-03
  9.81815974e-04 9.49195400e-03 3.37776504e-02 1.74738131e-02
  1.5380

[[7.82363641e-05 8.77744332e-03 3.70791113e-08 7.54344214e-08
  6.54421228e-08 9.45765805e-07 2.97918916e-04 1.49121695e-06
  2.46765067e-05 7.01441596e-12 3.16721344e-06 6.87104151e-10
  5.95629281e-05 3.50487826e-05 1.71304407e-06 1.54112356e-09
  3.86884669e-03 2.46196345e-04 6.38421843e-05 3.26254994e-05
  3.20641084e-05 4.70706820e-03 3.52929987e-04 2.44125910e-03
  8.93892684e-06 7.43253462e-12 2.14569494e-02 8.70540619e-12
  1.34821703e-05 1.13887360e-07 1.22335972e-02 3.85827560e-04
  9.64329753e-04 1.13492984e-04 5.96082769e-04 1.81262288e-03
  9.30000097e-05 4.24525875e-04 2.63472102e-05 1.20640070e-04
  7.24130718e-04 4.28505795e-04 2.73224141e-04 7.46859541e-06
  2.46181444e-04 1.08745496e-03 4.02396145e-05 2.40750451e-04
  5.20803369e-05 4.73025963e-02 2.42867321e-03 4.73980290e-05
  6.51880284e-04 2.77264102e-04 2.24702409e-03 5.41827781e-03
  1.15908831e-02 7.51614943e-03 1.02854241e-03 1.52512803e-04
  3.54528805e-04 1.61788287e-03 2.02923338e-03 8.74856934e-02
  1.7052

[[7.78701797e-05 1.21413141e-01 3.20005320e-08 2.38690973e-06
  3.62266223e-06 2.10278849e-05 4.81939409e-04 1.59314150e-05
  2.12057785e-04 1.56784927e-06 1.93019491e-06 1.90000196e-06
  2.98644125e-04 1.77822221e-05 1.37392544e-05 1.59956983e-06
  8.36822484e-03 1.43233826e-02 2.16634580e-05 1.36562646e-03
  6.28863403e-04 1.39802741e-02 2.76432466e-03 6.55202568e-03
  1.50034437e-04 3.59180945e-07 2.14728690e-03 1.45173082e-08
  4.80738119e-04 1.46377088e-05 2.96821482e-02 9.74718202e-03
  1.60580695e-01 1.06645767e-02 3.36191995e-04 9.19340644e-03
  1.21729262e-03 2.54926924e-02 2.48829555e-03 4.77393949e-03
  1.78587008e-02 2.93431047e-04 2.11276906e-03 3.98965273e-03
  5.21397043e-04 4.94068908e-03 3.10599729e-02 2.78762523e-02
  7.74193089e-03 3.34854983e-02 2.73291338e-02 3.02515808e-03
  1.40559105e-02 8.62362236e-03 6.06806204e-02 1.78530216e-02
  6.56594858e-02 4.93851416e-02 3.87802422e-02 4.67186887e-03
  8.52485280e-03 2.53209248e-02 4.12877165e-02 8.59514158e-03
  9.4840

[[8.6120213e-04 1.9101182e-02 8.7241124e-06 6.4114315e-06 5.6684442e-04
  5.0724402e-06 4.3479586e-04 2.4147676e-06 2.4458630e-05 6.4640215e-10
  6.3802901e-05 1.7940195e-07 8.2162960e-04 5.6316794e-06 1.8972978e-07
  3.4009364e-07 2.2294796e-03 3.0568204e-04 5.7597022e-06 2.9482508e-02
  6.8900734e-03 2.6356089e-03 1.8365512e-04 1.4254840e-02 7.4205971e-05
  1.2753743e-05 6.8189162e-03 3.7259922e-06 6.1674957e-04 2.7908962e-05
  4.0889472e-02 7.4282796e-03 5.1582102e-02 4.0742750e-03 3.7891936e-04
  2.6246339e-02 7.8359237e-03 1.1307414e-02 1.5288212e-03 1.2469898e-06
  2.7053314e-03 1.2576178e-04 3.4790875e-03 3.8771136e-04 3.1533051e-05
  2.4566529e-02 1.0583094e-02 3.0371904e-02 5.8562605e-04 2.9309040e-02
  1.2500626e-01 4.4117901e-02 8.4622644e-02 8.2047857e-02 2.4445783e-02
  1.4519668e-02 2.8359359e-02 2.1466924e-02 1.7202891e-02 8.6215343e-03
  1.3000242e-05 1.2483283e-01 4.6745338e-02 4.7945505e-04 3.2080279e-05
  3.9147602e-05 7.6435666e-05 1.1237759e-06 4.6750483e-05 2.5894

...

[[9.7108808e-05 3.5215256e-01 7.1986524e-06 8.1122398e-06 2.4415943e-04
  1.1489304e-03 4.6880357e-02 3.8859362e-04 2.8657234e-06 2.1748166e-07
  4.4057450e-07 1.9244402e-08 3.9678413e-04 3.1208030e-05 5.5862569e-07
  1.8169176e-05 2.4357976e-03 2.3960305e-04 2.5081521e-05 1.4121105e-03
  2.0847430e-04 1.7261317e-04 6.6253945e-02 2.9759434e-05 2.8366631e-07
  3.7867068e-10 4.2699259e-05 1.0522258e-08 2.1692413e-06 7.9975907e-06
  2.9749833e-02 8.7395305e-04 6.9532206e-04 4.0091742e-05 9.5729811e-05
  6.1499921e-04 9.6999400e-05 4.7070728e-04 1.6356299e-03 9.1422762e-06
  1.4723629e-02 2.7294316e-05 6.8865676e-04 2.1578200e-05 9.4455044e-04
  3.7413405e-03 1.4168662e-03 1.4146500e-03 2.6646056e-04 1.2528276e-03
  5.4838177e-04 3.6303038e-04 1.0490292e-03 1.4807411e-03 2.3013160e-01
  9.2296554e-03 2.1355324e-03 3.1774946e-02 3.7290975e-03 6.5588843e-05
  1.5218633e-04 2.7432689e-03 8.2173722e-04 1.3490088e-01 2.0355792e-03
  3.0192367e-03 1.3649117e-03 2.5016307e-03 6.5759605e-06 2.4669

[[3.56034107e-05 1.68380961e-01 2.92482127e-07 2.12180621e-05
  6.69878000e-06 3.01712455e-04 2.73913913e-03 1.44555033e-05
  2.29006182e-05 1.66654570e-06 5.84069312e-07 1.38534531e-07
  3.15785554e-04 5.73512466e-07 6.10465634e-08 1.56625356e-05
  9.45122854e-04 2.04206118e-03 2.06886889e-05 4.75309440e-04
  3.86193045e-04 4.32104629e-04 2.59241927e-02 1.02901750e-03
  2.93185337e-07 9.14888876e-09 6.46904868e-04 1.34810205e-10
  7.41579288e-06 2.52051359e-06 3.47514749e-02 8.84500507e-04
  1.47445267e-02 1.16358344e-04 2.24966614e-04 2.50915159e-03
  6.86771004e-04 6.34035654e-03 5.23214985e-04 5.02638472e-03
  4.53927629e-02 1.04943261e-04 2.12768675e-03 2.02908359e-05
  2.65170541e-03 1.54968143e-01 3.43884788e-02 1.15398634e-02
  5.76821668e-03 2.45927498e-02 6.98893564e-03 1.13923685e-03
  1.78213173e-03 3.61631298e-03 8.19113012e-03 1.88540727e-01
  1.31189138e-01 9.05375648e-03 7.24212453e-03 3.16026760e-03
  2.05531418e-02 1.03183044e-02 3.91385052e-03 2.22911630e-02
  3.9413

  4.07535481e-05 5.35170057e-06 1.19020471e-04]]
[[3.48668550e-06 1.40350489e-02 1.01816608e-08 1.36171502e-06
  2.01812072e-05 4.96633111e-05 5.60974411e-04 1.03351465e-06
  7.93228566e-04 8.18026180e-08 2.53270900e-06 5.73353987e-09
  9.64149058e-06 4.58376708e-05 1.08775830e-05 2.99191868e-07
  1.37303548e-03 1.02070756e-02 3.59843689e-05 1.53180840e-03
  2.58642598e-04 2.42597908e-02 3.22251435e-04 2.50302441e-03
  1.99036767e-05 2.85224466e-09 3.77840083e-03 3.19179655e-10
  4.81980300e-04 2.06846409e-04 5.38801067e-02 9.91888624e-03
  1.04952507e-01 3.03500332e-04 1.37617323e-03 4.42723092e-03
  6.40422630e-04 1.18950531e-02 7.93939515e-04 4.55050613e-04
  3.75905186e-02 1.02155325e-04 4.34859004e-03 1.22648999e-02
  3.11475829e-03 2.17885561e-02 1.52625272e-03 1.17801744e-02
  4.29102045e-04 2.29220111e-02 4.40755635e-02 1.17842469e-03
  9.26495437e-03 3.84683744e-03 6.46339133e-02 2.36465167e-02
  1.27665833e-01 1.77789688e-01 1.88779347e-02 3.94174783e-03
  1.18236255e-03 5.30

[[7.99941772e-04 4.97950707e-03 1.18385770e-12 3.43410761e-06
  4.20030988e-09 1.60056942e-07 1.25506431e-05 5.12924453e-06
  2.38004523e-05 9.64747726e-11 2.61403045e-07 1.28468308e-12
  1.68778279e-04 7.22969799e-06 2.51960923e-08 3.09101601e-14
  1.13751259e-04 1.19264955e-02 1.13450087e-05 6.72732294e-03
  7.14239385e-03 2.22311844e-03 4.51650430e-06 6.89395377e-03
  7.83408614e-05 2.67898442e-13 8.95517599e-03 1.31470067e-14
  3.33783355e-05 1.74373582e-07 1.14228344e-02 3.58077144e-04
  1.38184084e-02 2.11143342e-04 5.59427244e-05 2.23913137e-02
  1.38123743e-02 3.10786813e-02 1.11174304e-04 1.29484897e-03
  7.40938028e-03 2.96073500e-03 4.66333143e-03 6.47427863e-04
  1.89622864e-04 1.31156640e-02 2.72832834e-03 9.41411033e-03
  5.58061805e-03 2.97715422e-02 4.66402620e-02 8.95852037e-03
  3.88934240e-02 1.07827131e-02 1.48238987e-01 2.27442421e-02
  4.17436175e-02 1.22563438e-02 1.80254541e-02 9.72153433e-03
  9.95617709e-04 8.13883469e-02 1.64445758e-01 5.54390484e-03
  7.2194

[[5.4039083e-06 3.7166413e-02 6.7030055e-07 1.5408358e-04 1.1257246e-05
  1.1883475e-04 2.4859258e-03 2.1877929e-06 2.2210488e-04 3.4174663e-09
  4.0674186e-06 9.3673362e-09 3.2880380e-05 2.7508070e-06 7.8453786e-07
  1.0644153e-06 3.0801645e-03 2.9167153e-03 2.0253181e-02 7.8535726e-04
  1.1635626e-04 1.2599182e-03 1.2334083e-03 7.2841736e-05 1.4873997e-07
  3.7377877e-08 1.4194696e-03 4.3987476e-09 4.2522748e-05 2.2824049e-06
  4.2421248e-02 1.3201080e-03 8.7181065e-04 4.2199582e-04 2.2611224e-04
  1.1995373e-03 7.8860912e-03 4.3425374e-03 2.0423291e-05 3.0931269e-04
  1.4162572e-02 2.5126099e-04 1.5452881e-03 8.0892249e-05 7.1661745e-04
  6.2123942e-03 7.9146557e-04 2.0547633e-03 7.8248722e-04 2.4293402e-02
  1.5016062e-02 1.2402296e-03 3.1960295e-03 1.2451852e-03 3.6394227e-02
  1.6333437e-02 2.9352775e-02 5.7356834e-02 6.7748940e-03 1.9432385e-03
  2.3730984e-04 1.1000019e-02 3.4965570e-03 3.5922203e-02 9.7045757e-02
  9.7839618e-03 4.4966964e-03 1.8685759e-04 5.6347676e-04 7.6959

In [71]:
"".join(gen_data)

'म रौनियार उत्कृष्ट भएका छन् । समृद्धि फाउन्डेसनले आयोजना गरेको ‘उद्यमीको कथा, एक तस्बिर प्रतियोगिता’***                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [72]:
print(int_to_char)

{0: '\n', 1: ' ', 2: '!', 3: "'", 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '/', 10: '?', 11: 'm', 12: '\xa0', 13: '¥', 14: '«', 15: '÷', 16: 'ँ', 17: 'ं', 18: 'ः', 19: 'अ', 20: 'आ', 21: 'इ', 22: 'ई', 23: 'उ', 24: 'ऊ', 25: 'ऋ', 26: 'ए', 27: 'ऐ', 28: 'ओ', 29: 'औ', 30: 'क', 31: 'ख', 32: 'ग', 33: 'घ', 34: 'ङ', 35: 'च', 36: 'छ', 37: 'ज', 38: 'झ', 39: 'ञ', 40: 'ट', 41: 'ठ', 42: 'ड', 43: 'ढ', 44: 'ण', 45: 'त', 46: 'थ', 47: 'द', 48: 'ध', 49: 'न', 50: 'प', 51: 'फ', 52: 'ब', 53: 'भ', 54: 'म', 55: 'य', 56: 'र', 57: 'ल', 58: 'व', 59: 'श', 60: 'ष', 61: 'स', 62: 'ह', 63: 'ा', 64: 'ि', 65: 'ी', 66: 'ु', 67: 'ू', 68: 'ृ', 69: 'े', 70: 'ै', 71: 'ो', 72: 'ौ', 73: '्', 74: '।', 75: '०', 76: '१', 77: '२', 78: '३', 79: '४', 80: '५', 81: '६', 82: '७', 83: '८', 84: '९', 85: '\u200d', 86: '–', 87: '‘', 88: '’', 89: '“', 90: '”'}
